In [1]:
##### Carga de librerías y carga del dataset 'df_model_xgb_rf.csv' y eliminación de filas con valores faltantes

# Carga de librerías
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

# Carga de datos
df_model_rf = pd.read_csv(r"../data/final/1_panel/3_modelling/df_model_post_tratamiento_estadistico.csv")
df_model_rf = df_model_rf.dropna()
df_model_rf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24232 entries, 0 to 24231
Columns: 136 entries, status_inf to ratiodep
dtypes: bool(127), float64(4), int64(5)
memory usage: 4.6 MB


In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, roc_auc_score, recall_score, f1_score, precision_score, accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import numpy as np
import joblib

# 1. Definir dataset
X = df_model_rf.drop(columns=['status_inf'])
y = df_model_rf['status_inf']

# 2. Dividir los datos en 80% entrenamiento y 20% prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 3. Definir el modelo base Random Forest con los nuevos hiperparámetros
rf_model = RandomForestClassifier(
    n_estimators=300,          # Número de árboles
    max_depth=15,              # Profundidad máxima de los árboles
    max_features='sqrt',       # Número de características a considerar para cada árbol
    min_samples_leaf=2,        # Mínimo número de muestras en una hoja
    min_samples_split=5,       # Mínimo número de muestras para dividir un nodo
    random_state=42,
    n_jobs=-1,                 # Usar todos los núcleos disponibles
    class_weight='balanced',   # Ajustar los pesos de las clases para lidiar con el desbalance
)

# 4. Calcular el SMOTE
smote = SMOTE(random_state=42, sampling_strategy='auto')

# 5. Crear pipeline con SMOTE y Random Forest
pipeline = Pipeline(steps=[('smote', smote), ('rf', rf_model)])

# 6. Reducir el espacio de búsqueda de hiperparámetros
param_grid = {
    'rf__n_estimators': [300],  # Número de árboles
    'rf__max_depth': [15],      # Profundidad máxima
    'rf__min_samples_split': [5],  # Mínimo número de muestras para dividir un nodo
    'rf__min_samples_leaf': [2],  # Mínimo número de muestras en una hoja
    'rf__max_features': ['sqrt'],  # Número de características a considerar para cada árbol
}

# 7. Realizar la búsqueda en cuadrícula con validación cruzada en el conjunto de entrenamiento
grid_search = GridSearchCV(pipeline, param_grid, 
                           cv=StratifiedKFold(n_splits=4, shuffle=True, random_state=42),
                           scoring='roc_auc', n_jobs=1, verbose=2)

# 8. Ajustar el modelo con GridSearchCV en el conjunto de entrenamiento
grid_search.fit(X_train, y_train)

# 9. Obtener el mejor modelo de GridSearch
best_rf_model = grid_search.best_estimator_

# 10. Imprimir los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros encontrados: {grid_search.best_params_}")

# 11. Realizar las predicciones con el mejor modelo en el conjunto de prueba
y_pred = best_rf_model.predict(X_test)
y_proba_test = best_rf_model.predict_proba(X_test)[:, 1]

# 12. Aplicar umbral ajustado = 0.18 para las predicciones
y_pred_adjusted = (y_proba_test > 0.18).astype(int)

# 13. Reporte de clasificación en el conjunto de prueba
print("=== Reporte de Clasificación en el Conjunto de Prueba (umbral fijo = 0.18) ===")
print(classification_report(y_test, y_pred_adjusted, target_names=["Clase 0", "Clase 1"], digits=4))

# 14. Calcular AUC en el conjunto de prueba
auc = roc_auc_score(y_test, y_proba_test)
print(f"AUC en conjunto de prueba: {auc:.3f}")

# 15. Calcular otras métricas en el conjunto de prueba
accuracy = accuracy_score(y_test, y_pred_adjusted)
precision_class_1 = precision_score(y_test, y_pred_adjusted, pos_label=1)
recall_class_1 = recall_score(y_test, y_pred_adjusted, pos_label=1)
f1_class_1 = f1_score(y_test, y_pred_adjusted, pos_label=1)

# 16. Mostrar las métricas de evaluación en el conjunto de prueba
print(f"Accuracy: {accuracy:.3f}")
print(f"Precision (Clase 1): {precision_class_1:.3f}")
print(f"Recall (Clase 1): {recall_class_1:.3f}")
print(f"F1-Score (Clase 1): {f1_class_1:.3f}")

# 17. Guardar el modelo entrenado
joblib.dump(best_rf_model, '../output/modelos/RandomForest_model_optimized_with_SMOTE_and_gridsearch.joblib')


Fitting 4 folds for each of 1 candidates, totalling 4 fits
[CV] END rf__max_depth=15, rf__max_features=sqrt, rf__min_samples_leaf=2, rf__min_samples_split=5, rf__n_estimators=300; total time=   5.3s
[CV] END rf__max_depth=15, rf__max_features=sqrt, rf__min_samples_leaf=2, rf__min_samples_split=5, rf__n_estimators=300; total time=   1.5s
[CV] END rf__max_depth=15, rf__max_features=sqrt, rf__min_samples_leaf=2, rf__min_samples_split=5, rf__n_estimators=300; total time=   1.5s
[CV] END rf__max_depth=15, rf__max_features=sqrt, rf__min_samples_leaf=2, rf__min_samples_split=5, rf__n_estimators=300; total time=   1.4s
Mejores hiperparámetros encontrados: {'rf__max_depth': 15, 'rf__max_features': 'sqrt', 'rf__min_samples_leaf': 2, 'rf__min_samples_split': 5, 'rf__n_estimators': 300}
=== Reporte de Clasificación en el Conjunto de Prueba (umbral fijo = 0.18) ===
              precision    recall  f1-score   support

     Clase 0     0.9496    0.3353    0.4956      3991
     Clase 1     0.2283   

['../output/modelos/RandomForest_model_optimized_with_SMOTE_and_gridsearch.joblib']

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict, StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score, recall_score, f1_score, precision_score, accuracy_score
import numpy as np
import joblib

# 1. Definir dataset
X = df_model_rf.drop(columns=['status_inf'])
y = df_model_rf['status_inf']

# 2. Definir el modelo base Random Forest con ajuste de hiperparámetros
rf_model = RandomForestClassifier(
    random_state=42,
    n_jobs=-1,  # Usar todos los núcleos disponibles para el entrenamiento
    class_weight='balanced',  # Ajustar los pesos de las clases para lidiar con el desbalance
)

# 3. Reducir el espacio de búsqueda de hiperparámetros (para mantener < 300 fits)
param_grid = {
    'n_estimators': [100, 200, 300],          # Número de árboles (aumentado)
    'max_depth': [5, 10, 15],                 # Profundidad máxima de los árboles (ajustada)
    'min_samples_split': [2, 5],               # Mínimo número de muestras para dividir un nodo
    'min_samples_leaf': [1, 2, 5],             # Mínimo número de muestras en una hoja
    'max_features': ['sqrt', 'log2'],          # Número de características a considerar para cada árbol
}

# 4. Realizar la búsqueda en cuadrícula con validación cruzada
grid_search = GridSearchCV(rf_model, param_grid, 
                           cv=StratifiedKFold(n_splits=4, shuffle=True, random_state=42),
                           scoring='roc_auc', n_jobs=1, verbose=2)

# 5. Ajustar el modelo con GridSearchCV
grid_search.fit(X, y)

# 6. Obtener el mejor modelo de GridSearch
best_rf_model = grid_search.best_estimator_

# 7. Imprimir los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros encontrados: {grid_search.best_params_}")

# 8. Obtener las predicciones de probabilidad con el mejor modelo
y_proba_cv = cross_val_predict(best_rf_model, X, y, cv=StratifiedKFold(n_splits=4, shuffle=True, random_state=42), method='predict_proba')

# 9. Aplicar umbral ajustado = 0.18
y_pred_adjusted_cv = (y_proba_cv[:, 1] > 0.18).astype(int)

# 10. Reporte de clasificación con las predicciones ajustadas
print("=== Reporte de Clasificación con Validación Cruzada (umbral fijo = 0.18) ===")
print(classification_report(y, y_pred_adjusted_cv, target_names=["Clase 0", "Clase 1"], digits=4))

# 11. Inicializar listas para almacenar las métricas de cada pliegue
auc_cv_scores = []
accuracy_cv_scores = []
precision_class_1_cv_scores = []  # Guardaremos la precision para la clase 1
precision_macro_cv_scores = []  # Guardaremos la precision macro (promedio)
recall_class_1_cv_scores = []  # Guardaremos el recall para la clase 1
recall_macro_cv_scores = []  # Guardaremos el recall macro (promedio)
f1_class_1_cv_scores = []  # F1-Score para clase 1
f1_macro_cv_scores = []  # F1-Score macro
f1_cv_scores = []
recall_cv_scores = []  # Se agregó esta línea para definir la lista

cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
for train_idx, test_idx in cv.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    # Entrenar el mejor modelo en el pliegue de entrenamiento
    best_rf_model.fit(X_train, y_train)
    
    # Obtener las probabilidades predichas para AUC
    y_proba_test = best_rf_model.predict_proba(X_test)[:, 1]
    
    # Calcular AUC para este pliegue
    auc_cv_scores.append(roc_auc_score(y_test, y_proba_test))
    
    # Obtener las predicciones para métricas de clasificación
    y_pred_test = (y_proba_test > 0.18).astype(int)
    
    # Calcular las métricas para la clase 1
    precision_class_1 = precision_score(y_test, y_pred_test, pos_label=1)
    recall_class_1 = recall_score(y_test, y_pred_test, pos_label=1)
    f1_class_1 = f1_score(y_test, y_pred_test, pos_label=1)

    precision_class_1_cv_scores.append(precision_class_1)
    recall_class_1_cv_scores.append(recall_class_1)
    f1_class_1_cv_scores.append(f1_class_1)

    # Calcular Precision macro (promedio)
    precision_macro = precision_score(y_test, y_pred_test, average='macro')
    precision_macro_cv_scores.append(precision_macro)
    
    # Calcular Recall macro (promedio)
    recall_macro = recall_score(y_test, y_pred_test, average='macro')
    recall_macro_cv_scores.append(recall_macro)
    
    # Calcular F1-Score macro (promedio)
    f1_macro = f1_score(y_test, y_pred_test, average='macro')
    f1_macro_cv_scores.append(f1_macro)
    
    # Calcular F1-Score general
    f1_cv_scores.append(f1_score(y_test, y_pred_test))
    
    # Calcular Recall general
    recall_cv_scores.append(recall_score(y_test, y_pred_test))

# 12. Calcular la media y desviación estándar de las métricas
mean_auc = np.mean(auc_cv_scores)
std_auc = np.std(auc_cv_scores)

mean_accuracy = np.mean(accuracy_cv_scores)
std_accuracy = np.std(accuracy_cv_scores)

mean_precision_class_1 = np.mean(precision_class_1_cv_scores)
std_precision_class_1 = np.std(precision_class_1_cv_scores)

mean_precision_macro = np.mean(precision_macro_cv_scores)
std_precision_macro = np.std(precision_macro_cv_scores)

mean_recall_class_1 = np.mean(recall_class_1_cv_scores)
std_recall_class_1 = np.std(recall_class_1_cv_scores)

mean_recall_macro = np.mean(recall_macro_cv_scores)
std_recall_macro = np.std(recall_macro_cv_scores)

mean_f1_class_1 = np.mean(f1_class_1_cv_scores)
std_f1_class_1 = np.std(f1_class_1_cv_scores)

mean_f1_macro = np.mean(f1_macro_cv_scores)
std_f1_macro = np.std(f1_macro_cv_scores)

mean_f1 = np.mean(f1_cv_scores)
std_f1 = np.std(f1_cv_scores)

# 13. Imprimir los resultados finales con las métricas y sus desviaciones estándar
print(f"\nAUC promedio en validación cruzada: {mean_auc:.3f} ({std_auc:.3f})")
print(f"Accuracy promedio en validación cruzada: {mean_accuracy:.3f} ({std_accuracy:.3f})")
print(f"Precision promedio para la clase 1 en validación cruzada: {mean_precision_class_1:.3f} ({std_precision_class_1:.3f})")
print(f"Precision promedio macro en validación cruzada: {mean_precision_macro:.3f} ({std_precision_macro:.3f})")
print(f"Recall promedio para la clase 1 en validación cruzada: {mean_recall_class_1:.3f} ({std_recall_class_1:.3f})")
print(f"Recall promedio macro en validación cruzada: {mean_recall_macro:.3f} ({std_recall_macro:.3f})")
print(f"F1-score promedio para la clase 1 en validación cruzada: {mean_f1_class_1:.3f} ({std_f1_class_1:.3f})")
print(f"F1-score promedio macro en validación cruzada: {mean_f1_macro:.3f} ({std_f1_macro:.3f})")
print(f"F1-score promedio en validación cruzada: {mean_f1:.3f} ({std_f1:.3f})")

# 14. Guardar el modelo entrenado en la ruta especificada
joblib.dump(best_rf_model, '../output/modelos/RandomForest_model_optimized_with_gridsearch.joblib')


Fitting 4 folds for each of 108 candidates, totalling 432 fits
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.4s
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.7s
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.6s
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.7s
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; tota

c:\Users\sergio.cuadros\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\sergio.cuadros\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\sergio.cuadros\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\_methods.py:222: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\sergio.cuadros\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\_methods.py:180: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\sergio.cuadros\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\_methods.py:214: RuntimeWarning: invalid value encountered

['../output/modelos/RandomForest_model_optimized_with_gridsearch.joblib']

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict, StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score, recall_score, f1_score, precision_score, accuracy_score
import numpy as np
import joblib

# 1. Definir dataset
X = df_model_rf.drop(columns=['status_inf'])
y = df_model_rf['status_inf']

# 2. Definir el modelo base Random Forest con ajuste de hiperparámetros
rf_model = RandomForestClassifier(
    random_state=42,
    n_jobs=-1,  # Usar todos los núcleos disponibles para el entrenamiento
    class_weight='balanced',  # Ajustar los pesos de las clases para lidiar con el desbalance
)

# 3. Reducir el espacio de búsqueda de hiperparámetros (para mantener < 300 fits)
param_grid = {
    'n_estimators': [100, 200, 300],          # Número de árboles (aumentado)
    'max_depth': [5, 10, 15],                 # Profundidad máxima de los árboles (ajustada)
    'min_samples_split': [2, 5],               # Mínimo número de muestras para dividir un nodo
    'min_samples_leaf': [1, 2, 5],             # Mínimo número de muestras en una hoja
    'max_features': ['sqrt', 'log2'],          # Número de características a considerar para cada árbol
}

# 4. Realizar la búsqueda en cuadrícula con validación cruzada
grid_search = GridSearchCV(rf_model, param_grid, 
                           cv=StratifiedKFold(n_splits=4, shuffle=True, random_state=42),
                           scoring='roc_auc', n_jobs=1, verbose=2)

# 5. Ajustar el modelo con GridSearchCV
grid_search.fit(X, y)

# 6. Obtener el mejor modelo de GridSearch
best_rf_model = grid_search.best_estimator_

# 7. Obtener las predicciones de probabilidad con el mejor modelo
y_proba_cv = cross_val_predict(best_rf_model, X, y, cv=StratifiedKFold(n_splits=4, shuffle=True, random_state=42), method='predict_proba')

# 8. Aplicar umbral ajustado = 0.18
y_pred_adjusted_cv = (y_proba_cv[:, 1] > 0.18).astype(int)

# 9. Reporte de clasificación con las predicciones ajustadas
print("=== Reporte de Clasificación con Validación Cruzada (umbral fijo = 0.18) ===")
print(classification_report(y, y_pred_adjusted_cv, target_names=["Clase 0", "Clase 1"], digits=4))

# 10. Inicializar listas para almacenar las métricas de cada pliegue
auc_cv_scores = []
accuracy_cv_scores = []
precision_class_1_cv_scores = []  # Guardaremos la precision para la clase 1
precision_macro_cv_scores = []  # Guardaremos la precision macro (promedio)
recall_class_1_cv_scores = []  # Guardaremos el recall para la clase 1
recall_macro_cv_scores = []  # Guardaremos el recall macro (promedio)
f1_class_1_cv_scores = []  # F1-Score para clase 1
f1_macro_cv_scores = []  # F1-Score macro
f1_cv_scores = []
recall_cv_scores = []  # Se agregó esta línea para definir la lista

cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
for train_idx, test_idx in cv.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    # Entrenar el mejor modelo en el pliegue de entrenamiento
    best_rf_model.fit(X_train, y_train)
    
    # Obtener las probabilidades predichas para AUC
    y_proba_test = best_rf_model.predict_proba(X_test)[:, 1]
    
    # Calcular AUC para este pliegue
    auc_cv_scores.append(roc_auc_score(y_test, y_proba_test))
    
    # Obtener las predicciones para métricas de clasificación
    y_pred_test = (y_proba_test > 0.18).astype(int)
    
    # Calcular las métricas para la clase 1
    precision_class_1 = precision_score(y_test, y_pred_test, pos_label=1)
    recall_class_1 = recall_score(y_test, y_pred_test, pos_label=1)
    f1_class_1 = f1_score(y_test, y_pred_test, pos_label=1)

    precision_class_1_cv_scores.append(precision_class_1)
    recall_class_1_cv_scores.append(recall_class_1)
    f1_class_1_cv_scores.append(f1_class_1)

    # Calcular Precision macro (promedio)
    precision_macro = precision_score(y_test, y_pred_test, average='macro')
    precision_macro_cv_scores.append(precision_macro)
    
    # Calcular Recall macro (promedio)
    recall_macro = recall_score(y_test, y_pred_test, average='macro')
    recall_macro_cv_scores.append(recall_macro)
    
    # Calcular F1-Score macro (promedio)
    f1_macro = f1_score(y_test, y_pred_test, average='macro')
    f1_macro_cv_scores.append(f1_macro)
    
    # Calcular F1-Score general
    f1_cv_scores.append(f1_score(y_test, y_pred_test))
    
    # Calcular Recall general
    recall_cv_scores.append(recall_score(y_test, y_pred_test))

# 11. Calcular la media y desviación estándar de las métricas
mean_auc = np.mean(auc_cv_scores)
std_auc = np.std(auc_cv_scores)

mean_accuracy = np.mean(accuracy_cv_scores)
std_accuracy = np.std(accuracy_cv_scores)

mean_precision_class_1 = np.mean(precision_class_1_cv_scores)
std_precision_class_1 = np.std(precision_class_1_cv_scores)

mean_precision_macro = np.mean(precision_macro_cv_scores)
std_precision_macro = np.std(precision_macro_cv_scores)

mean_recall_class_1 = np.mean(recall_class_1_cv_scores)
std_recall_class_1 = np.std(recall_class_1_cv_scores)

mean_recall_macro = np.mean(recall_macro_cv_scores)
std_recall_macro = np.std(recall_macro_cv_scores)

mean_f1_class_1 = np.mean(f1_class_1_cv_scores)
std_f1_class_1 = np.std(f1_class_1_cv_scores)

mean_f1_macro = np.mean(f1_macro_cv_scores)
std_f1_macro = np.std(f1_macro_cv_scores)

mean_f1 = np.mean(f1_cv_scores)
std_f1 = np.std(f1_cv_scores)

# 12. Imprimir los resultados finales con las métricas y sus desviaciones estándar
print(f"\nAUC promedio en validación cruzada: {mean_auc:.3f} ({std_auc:.3f})")
print(f"Accuracy promedio en validación cruzada: {mean_accuracy:.3f} ({std_accuracy:.3f})")
print(f"Precision promedio para la clase 1 en validación cruzada: {mean_precision_class_1:.3f} ({std_precision_class_1:.3f})")
print(f"Precision promedio macro en validación cruzada: {mean_precision_macro:.3f} ({std_precision_macro:.3f})")
print(f"Recall promedio para la clase 1 en validación cruzada: {mean_recall_class_1:.3f} ({std_recall_class_1:.3f})")
print(f"Recall promedio macro en validación cruzada: {mean_recall_macro:.3f} ({std_recall_macro:.3f})")
print(f"F1-score promedio para la clase 1 en validación cruzada: {mean_f1_class_1:.3f} ({std_f1_class_1:.3f})")
print(f"F1-score promedio macro en validación cruzada: {mean_f1_macro:.3f} ({std_f1_macro:.3f})")
print(f"F1-score promedio en validación cruzada: {mean_f1:.3f} ({std_f1:.3f})")

# 13. Guardar el modelo entrenado en la ruta especificada
joblib.dump(best_rf_model, '../output/modelos/RandomForest_model_optimized_with_gridsearch.joblib')


Fitting 4 folds for each of 108 candidates, totalling 432 fits
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.4s
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.4s
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.7s
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.9s
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.9s
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; tota

c:\Users\sergio.cuadros\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\sergio.cuadros\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\sergio.cuadros\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\_methods.py:222: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\sergio.cuadros\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\_methods.py:180: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\sergio.cuadros\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\_methods.py:214: RuntimeWarning: invalid value encountered


AUC promedio en validación cruzada: 0.741 (0.005)
Accuracy promedio en validación cruzada: nan (nan)
Precision promedio para la clase 1 en validación cruzada: 0.204 (0.001)
Precision promedio macro en validación cruzada: 0.589 (0.001)
Recall promedio para la clase 1 en validación cruzada: 0.976 (0.002)
Recall promedio macro en validación cruzada: 0.580 (0.002)
F1-score promedio para la clase 1 en validación cruzada: 0.338 (0.001)
F1-score promedio macro en validación cruzada: 0.324 (0.004)
F1-score promedio en validación cruzada: 0.338 (0.001)


['../output/modelos/RandomForest_model_optimized_with_gridsearch.joblib']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict, StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score, f1_score, recall_score, precision_score, accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
import numpy as np
import joblib

# 1. Definir dataset
X = df_model_rf.drop(columns=['status_inf'])
y = df_model_rf['status_inf']

# 2. Definir el modelo base Random Forest con ajuste de hiperparámetros
rf_model = RandomForestClassifier(
    random_state=42,
    n_jobs=-1,  # Usar todos los núcleos disponibles para el entrenamiento
    class_weight='balanced',  # Ajustar los pesos de las clases para lidiar con el desbalance
)

# 3. Calcular el SMOTE
smote = SMOTE(random_state=42, sampling_strategy='auto')

# 4. Crear pipeline con SMOTE y Random Forest
pipeline = Pipeline(steps=[('smote', smote), ('rf', rf_model)])

# 5. Reducir el espacio de búsqueda de hiperparámetros (para mantener < 300 fits)
param_grid = {
    'rf__n_estimators': [100, 200, 300],          # Número de árboles (aumentado)
    'rf__max_depth': [5, 10, 15],                 # Profundidad máxima de los árboles (ajustada)
    'rf__min_samples_split': [2, 5],               # Mínimo número de muestras para dividir un nodo
    'rf__min_samples_leaf': [1, 2, 5],             # Mínimo número de muestras en una hoja
    'rf__max_features': ['sqrt', 'log2'],          # Número de características a considerar para cada árbol
}

# 6. Realizar la búsqueda en cuadrícula con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, 
                           cv=StratifiedKFold(n_splits=4, shuffle=True, random_state=42),
                           scoring='roc_auc', n_jobs=1, verbose=2)

# 7. Ajustar el modelo con GridSearchCV
grid_search.fit(X, y)

# 8. Obtener el mejor modelo de GridSearch
best_rf_model = grid_search.best_estimator_

# 9. Obtener las predicciones de probabilidad con el mejor modelo
y_proba_cv = cross_val_predict(best_rf_model, X, y, cv=StratifiedKFold(n_splits=4, shuffle=True, random_state=42), method='predict_proba')

# 10. Aplicar umbral ajustado = 0.18
y_pred_adjusted_cv = (y_proba_cv[:, 1] > 0.18).astype(int)

# 11. Reporte de clasificación con las predicciones ajustadas
print("=== Reporte de Clasificación con Validación Cruzada (umbral fijo = 0.18) ===")
print(classification_report(y, y_pred_adjusted_cv, target_names=["Clase 0", "Clase 1"], digits=4))

# 12. Inicializar listas para almacenar las métricas de cada pliegue
auc_cv_scores = []
accuracy_cv_scores = []
precision_class_1_cv_scores = []  # Guardaremos la precision para la clase 1
precision_macro_cv_scores = []  # Guardaremos la precision macro (promedio)
recall_class_1_cv_scores = []  # Guardaremos el recall para la clase 1
recall_macro_cv_scores = []  # Guardaremos el recall macro (promedio)
f1_class_1_cv_scores = []  # F1-Score para clase 1
f1_macro_cv_scores = []  # F1-Score macro
f1_cv_scores = []

cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
for train_idx, test_idx in cv.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    # Entrenar el mejor modelo en el pliegue de entrenamiento
    best_rf_model.fit(X_train, y_train)
    
    # Obtener las probabilidades predichas para AUC
    y_proba_test = best_rf_model.predict_proba(X_test)[:, 1]
    
    # Calcular AUC para este pliegue
    auc_cv_scores.append(roc_auc_score(y_test, y_proba_test))
    
    # Obtener las predicciones para métricas de clasificación
    y_pred_test = (y_proba_test > 0.18).astype(int)
    
    # Calcular Precision para la clase 1
    precision_class_1 = precision_score(y_test, y_pred_test, pos_label=1)
    precision_class_1_cv_scores.append(precision_class_1)
    
    # Calcular Precision macro (promedio)
    precision_macro = precision_score(y_test, y_pred_test, average='macro')
    precision_macro_cv_scores.append(precision_macro)
    
    # Calcular Recall para la clase 1
    recall_class_1 = recall_score(y_test, y_pred_test, pos_label=1)
    recall_class_1_cv_scores.append(recall_class_1)
    
    # Calcular Recall macro (promedio)
    recall_macro = recall_score(y_test, y_pred_test, average='macro')
    recall_macro_cv_scores.append(recall_macro)
    
    # Calcular F1-Score para clase 1
    f1_class_1 = f1_score(y_test, y_pred_test, pos_label=1)
    f1_class_1_cv_scores.append(f1_class_1)
    
    # Calcular F1-Score macro (promedio)
    f1_macro = f1_score(y_test, y_pred_test, average='macro')
    f1_macro_cv_scores.append(f1_macro)
    
    # Calcular F1-Score general
    f1_cv_scores.append(f1_score(y_test, y_pred_test))

# 13. Calcular la media y desviación estándar de las métricas
mean_auc = np.mean(auc_cv_scores)
std_auc = np.std(auc_cv_scores)

mean_accuracy = np.mean(accuracy_cv_scores)
std_accuracy = np.std(accuracy_cv_scores)

mean_precision_class_1 = np.mean(precision_class_1_cv_scores)
std_precision_class_1 = np.std(precision_class_1_cv_scores)

mean_precision_macro = np.mean(precision_macro_cv_scores)
std_precision_macro = np.std(precision_macro_cv_scores)

mean_recall_class_1 = np.mean(recall_class_1_cv_scores)
std_recall_class_1 = np.std(recall_class_1_cv_scores)

mean_recall_macro = np.mean(recall_macro_cv_scores)
std_recall_macro = np.std(recall_macro_cv_scores)

mean_f1_class_1 = np.mean(f1_class_1_cv_scores)
std_f1_class_1 = np.std(f1_class_1_cv_scores)

mean_f1_macro = np.mean(f1_macro_cv_scores)
std_f1_macro = np.std(f1_macro_cv_scores)

mean_f1 = np.mean(f1_cv_scores)
std_f1 = np.std(f1_cv_scores)

# 14. Imprimir resultados
print(f"\nAUC promedio en validación cruzada: {mean_auc:.3f} ({std_auc:.3f})")
print(f"Accuracy promedio en validación cruzada: {mean_accuracy:.3f} ({std_accuracy:.3f})")
print(f"Precision promedio para la clase 1 en validación cruzada: {mean_precision_class_1:.3f} ({std_precision_class_1:.3f})")
print(f"Precision promedio macro en validación cruzada: {mean_precision_macro:.3f} ({std_precision_macro:.3f})")
print(f"Recall promedio para la clase 1 en validación cruzada: {mean_recall_class_1:.3f} ({std_recall_class_1:.3f})")
print(f"Recall promedio macro en validación cruzada: {mean_recall_macro:.3f} ({std_recall_macro:.3f})")
print(f"F1-score promedio para la clase 1 en validación cruzada: {mean_f1_class_1:.3f} ({std_f1_class_1:.3f})")
print(f"F1-score promedio macro en validación cruzada: {mean_f1_macro:.3f} ({std_f1_macro:.3f})")
print(f"F1-score promedio en validación cruzada: {mean_f1:.3f} ({std_f1:.3f})")

# 15. Guardar el modelo entrenado en la ruta especificada
joblib.dump(best_rf_model, '../output/modelos/RandomForest_model_optimized_with_SMOTE_and_gridsearch.joblib')


Fitting 4 folds for each of 108 candidates, totalling 432 fits
[CV] END rf__max_depth=5, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=100; total time=   4.2s
[CV] END rf__max_depth=5, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=100; total time=   0.8s
[CV] END rf__max_depth=5, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=100; total time=   0.8s
[CV] END rf__max_depth=5, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=100; total time=   0.8s
[CV] END rf__max_depth=5, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=200; total time=   1.3s
[CV] END rf__max_depth=5, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=200; total time=   1.3s
[CV] END rf__max_depth=5, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n

c:\Users\sergio.cuadros\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\sergio.cuadros\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\sergio.cuadros\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\_methods.py:222: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\sergio.cuadros\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\_methods.py:180: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\sergio.cuadros\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\_methods.py:214: RuntimeWarning: invalid value encountered


AUC promedio en validación cruzada: 0.722 (0.009)
Accuracy promedio en validación cruzada: nan (nan)
Precision promedio para la clase 1 en validación cruzada: 0.228 (0.001)
Precision promedio macro en validación cruzada: 0.590 (0.002)
Recall promedio para la clase 1 en validación cruzada: 0.920 (0.009)
Recall promedio macro en validación cruzada: 0.626 (0.002)
F1-score promedio para la clase 1 en validación cruzada: 0.366 (0.001)
F1-score promedio macro en validación cruzada: 0.430 (0.006)
F1-score promedio en validación cruzada: 0.366 (0.001)


['../output/modelos/RandomForest_model_optimized_with_SMOTE_and_gridsearch.joblib']

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict, StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score, recall_score, f1_score, precision_score, accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
import numpy as np
import joblib

# 1. Definir dataset
X = df_model_rf.drop(columns=['status_inf'])
y = df_model_rf['status_inf']

# 2. Definir el modelo base Random Forest con ajuste de hiperparámetros
rf_model = RandomForestClassifier(
    random_state=42,
    n_jobs=-1,  # Usar todos los núcleos disponibles para el entrenamiento
    class_weight='balanced',  # Ajustar los pesos de las clases para lidiar con el desbalance
)

# 3. Calcular el SMOTE
smote = SMOTE(random_state=42, sampling_strategy='auto')

# 4. Crear pipeline con SMOTE y Random Forest
pipeline = Pipeline(steps=[('smote', smote), ('rf', rf_model)])

# 5. Reducir el espacio de búsqueda de hiperparámetros (para mantener < 300 fits)
param_grid = {
    'rf__n_estimators': [100, 200, 300],          # Número de árboles (aumentado)
    'rf__max_depth': [5, 10, 15],                 # Profundidad máxima de los árboles (ajustada)
    'rf__min_samples_split': [2, 5],               # Mínimo número de muestras para dividir un nodo
    'rf__min_samples_leaf': [1, 2, 5],             # Mínimo número de muestras en una hoja
    'rf__max_features': ['sqrt', 'log2'],          # Número de características a considerar para cada árbol
}

# 6. Realizar la búsqueda en cuadrícula con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, 
                           cv=StratifiedKFold(n_splits=4, shuffle=True, random_state=42),
                           scoring='roc_auc', n_jobs=1, verbose=2)

# 7. Ajustar el modelo con GridSearchCV
grid_search.fit(X, y)

# 8. Obtener el mejor modelo de GridSearch
best_rf_model = grid_search.best_estimator_

# 9. Imprimir los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros encontrados: {grid_search.best_params_}")

# 10. Obtener las predicciones de probabilidad con el mejor modelo
y_proba_cv = cross_val_predict(best_rf_model, X, y, cv=StratifiedKFold(n_splits=4, shuffle=True, random_state=42), method='predict_proba')

# 11. Aplicar umbral ajustado = 0.18
y_pred_adjusted_cv = (y_proba_cv[:, 1] > 0.18).astype(int)

# 12. Reporte de clasificación con las predicciones ajustadas
print("=== Reporte de Clasificación con Validación Cruzada (umbral fijo = 0.18) ===")
print(classification_report(y, y_pred_adjusted_cv, target_names=["Clase 0", "Clase 1"], digits=4))

# 13. Inicializar listas para almacenar las métricas de cada pliegue
auc_cv_scores = []
accuracy_cv_scores = []
precision_class_1_cv_scores = []  # Guardamos la precision para la clase 1
precision_macro_cv_scores = []  # Guardamos la precision macro (promedio)
recall_class_1_cv_scores = []  # Guardamos el recall para la clase 1
recall_macro_cv_scores = []  # Guardamos el recall macro (promedio)
f1_class_1_cv_scores = []  # F1-Score para clase 1
f1_macro_cv_scores = []  # F1-Score macro
f1_cv_scores = []

cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
for train_idx, test_idx in cv.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    # Entrenar el mejor modelo en el pliegue de entrenamiento
    best_rf_model.fit(X_train, y_train)
    
    # Obtener las probabilidades predichas para AUC
    y_proba_test = best_rf_model.predict_proba(X_test)[:, 1]
    
    # Calcular AUC para este pliegue
    auc_cv_scores.append(roc_auc_score(y_test, y_proba_test))
    
    # Obtener las predicciones para métricas de clasificación
    y_pred_test = (y_proba_test > 0.18).astype(int)
    
    # Calcular las métricas para la clase 1
    precision_class_1 = precision_score(y_test, y_pred_test, pos_label=1)
    recall_class_1 = recall_score(y_test, y_pred_test, pos_label=1)
    f1_class_1 = f1_score(y_test, y_pred_test, pos_label=1)

    precision_class_1_cv_scores.append(precision_class_1)
    recall_class_1_cv_scores.append(recall_class_1)
    f1_class_1_cv_scores.append(f1_class_1)

    # Calcular Precision macro (promedio)
    precision_macro = precision_score(y_test, y_pred_test, average='macro')
    precision_macro_cv_scores.append(precision_macro)
    
    # Calcular Recall macro (promedio)
    recall_macro = recall_score(y_test, y_pred_test, average='macro')
    recall_macro_cv_scores.append(recall_macro)
    
    # Calcular F1-Score macro (promedio)
    f1_macro = f1_score(y_test, y_pred_test, average='macro')
    f1_macro_cv_scores.append(f1_macro)
    
    # Calcular F1-Score general
    f1_cv_scores.append(f1_score(y_test, y_pred_test))
    
    # Calcular Recall general
    recall_cv_scores.append(recall_score(y_test, y_pred_test))

# 14. Calcular la media y desviación estándar de las métricas
mean_auc = np.mean(auc_cv_scores)
std_auc = np.std(auc_cv_scores)

mean_accuracy = np.mean(accuracy_cv_scores)
std_accuracy = np.std(accuracy_cv_scores)

mean_precision_class_1 = np.mean(precision_class_1_cv_scores)
std_precision_class_1 = np.std(precision_class_1_cv_scores)

mean_precision_macro = np.mean(precision_macro_cv_scores)
std_precision_macro = np.std(precision_macro_cv_scores)

mean_recall_class_1 = np.mean(recall_class_1_cv_scores)
std_recall_class_1 = np.std(recall_class_1_cv_scores)

mean_recall_macro = np.mean(recall_macro_cv_scores)
std_recall_macro = np.std(recall_macro_cv_scores)

mean_f1_class_1 = np.mean(f1_class_1_cv_scores)
std_f1_class_1 = np.std(f1_class_1_cv_scores)

mean_f1_macro = np.mean(f1_macro_cv_scores)
std_f1_macro = np.std(f1_macro_cv_scores)

mean_f1 = np.mean(f1_cv_scores)
std_f1 = np.std(f1_cv_scores)

# 15. Imprimir los resultados finales con las métricas y sus desviaciones estándar
print(f"\nAUC promedio en validación cruzada: {mean_auc:.3f} ({std_auc:.3f})")
print(f"Accuracy promedio en validación cruzada: {mean_accuracy:.3f} ({std_accuracy:.3f})")
print(f"Precision promedio para la clase 1 en validación cruzada: {mean_precision_class_1:.3f} ({std_precision_class_1:.3f})")
print(f"Precision promedio macro en validación cruzada: {mean_precision_macro:.3f} ({std_precision_macro:.3f})")
print(f"Recall promedio para la clase 1 en validación cruzada: {mean_recall_class_1:.3f} ({std_recall_class_1:.3f})")
print(f"Recall promedio macro en validación cruzada: {mean_recall_macro:.3f} ({std_recall_macro:.3f})")
print(f"F1-score promedio para la clase 1 en validación cruzada: {mean_f1_class_1:.3f} ({std_f1_class_1:.3f})")
print(f"F1-score promedio macro en validación cruzada: {mean_f1_macro:.3f} ({std_f1_macro:.3f})")
print(f"F1-score promedio en validación cruzada: {mean_f1:.3f} ({std_f1:.3f})")

# 16. Guardar el modelo entrenado en la ruta especificada
joblib.dump(best_rf_model, '../output/modelos/RandomForest_model_optimized_with_SMOTE_and_gridsearch.joblib')


Fitting 4 folds for each of 108 candidates, totalling 432 fits
[CV] END rf__max_depth=5, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=100; total time=   4.9s
[CV] END rf__max_depth=5, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=100; total time=   0.7s
[CV] END rf__max_depth=5, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=100; total time=   0.7s
[CV] END rf__max_depth=5, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=100; total time=   0.7s
[CV] END rf__max_depth=5, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=200; total time=   1.2s
[CV] END rf__max_depth=5, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=200; total time=   1.1s
[CV] END rf__max_depth=5, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n

c:\Users\sergio.cuadros\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\sergio.cuadros\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\sergio.cuadros\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\_methods.py:222: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\sergio.cuadros\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\_methods.py:180: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\sergio.cuadros\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\_methods.py:214: RuntimeWarning: invalid value encountered


AUC promedio en validación cruzada: 0.722 (0.009)
Accuracy promedio en validación cruzada: nan (nan)
Precision promedio para la clase 1 en validación cruzada: 0.228 (0.001)
Precision promedio macro en validación cruzada: 0.590 (0.002)
Recall promedio para la clase 1 en validación cruzada: 0.920 (0.009)
Recall promedio macro en validación cruzada: 0.626 (0.002)
F1-score promedio para la clase 1 en validación cruzada: 0.366 (0.001)
F1-score promedio macro en validación cruzada: 0.430 (0.006)
F1-score promedio en validación cruzada: 0.366 (0.001)


['../output/modelos/RandomForest_model_optimized_with_SMOTE_and_gridsearch.joblib']